In [1]:
import sys
from poker_env import PokerEnv
from agents.random_policy import RandomActions
from agents.heuristic_policy import HeuristicPolicy
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.ppo import (
    PPOConfig,
    PPOTF1Policy,
    PPOTF2Policy,
    PPOTorchPolicy,
)
from gym import spaces
import mpu
import numpy as np
import ray
from ray.rllib.models import MODEL_DEFAULTS
from ray.rllib.policy.policy import PolicySpec
from ray.tune.registry import register_env

/Users/adamprice/Applications/anaconda3/envs/stocktake/lib/python3.8/site-packages/keras_preprocessing/image/utils.py:23: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
/Users/adamprice/Applications/anaconda3/envs/stocktake/lib/python3.8/site-packages/keras_preprocessing/image/utils.py:24: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': pil_image.BILINEAR,
/Users/adamprice/Applications/anaconda3/envs/stocktake/lib/python3.8/site-packages/keras_preprocessing/image/utils.py:25: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  'bicubic': pil_image.BICUBIC,
/Users/adamprice/Applications/anaconda3/envs/stocktake/lib/python3.8/site-packages/keras_preprocessing/image/utils.py:28: DeprecationWarning: HAMMING is dep

In [2]:
from typing import Dict, Optional

class wrapped_poker_env(PokerEnv):
    def __init__(self, select_policy, config: Optional[Dict] = None):
        super().__init__(select_policy, config)
    
    def set_empty_obs(self):
        for a in self.players_ids:
            if 'Heuristic' in self.select_policy(self.agents[a].ID, 0):
                self.agents[a].empty_obs = {"hand": np.zeros(24), "community": np.zeros(24)}
            elif 'SimpleHand' in self.select_policy(self.agents[a].ID, 0):
                self.agents[a].empty_obs = {"obs": np.zeros(3+16+4),'state': np.zeros(1)}
            else:
                self.agents[a].empty_obs ={'obs': np.zeros(24+24+16+4), 'state': np.zeros(1)} 

    def get_obs(self, agent):
        community_cards_state = np.sum(self.community_cards, axis=0)
        if 'Heuristic' in self.select_policy(self.agents[agent].ID, 0):
            return {"hand": self.agents[agent].get_hand(), "community": community_cards_state} 
        elif 'SimpleHand' in self.select_policy(self.agents[agent].ID, 0):
            obs = np.array(self.score_hand(self.agents[agent].hand))
            obs = np.concatenate([obs, self.all_bets.reshape(-1)])
            obs = np.concatenate([obs, np.array([self.agents[a].chips for a in self.players_ids])])
            return {"obs": obs, "state": np.array([0])}
        
        obs = self.agents[agent].get_hand()
        obs = np.concatenate([obs, community_cards_state])
        obs = np.concatenate([obs, self.all_bets.reshape(-1)])
        obs = np.concatenate([obs, np.array([self.agents[a].chips for a in self.players_ids])])
        return {"obs": obs, "state": np.array([0])}




/Users/adamprice/Applications/anaconda3/envs/stocktake/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
def select_policy(agent_id, episode, **kwargs):
    if agent_id == 0:
        return "learned"
    elif agent_id == 1:
        return "SimpleHand"
    elif agent_id == 2:
        return "Heuristic_10"
    elif agent_id == 3:
        return "Heuristic_100"
    return "Heuristic_1000"

def env_creator(config):
    env = wrapped_poker_env(select_policy, config)
    return env

register_env("poker", lambda config: env_creator(config))

In [4]:
heuristic_observation_space = spaces.Dict({
            "hand": spaces.Box(0, 1, shape=(24, )),
            "community": spaces.Box(0, 1, shape=(24, ))
        })

simple_hand_observation_space = spaces.Dict({
            "obs": spaces.Box(0, 400, shape=(3+16+4, )),
            "state": spaces.Box(0, 1, shape=(1, ))
        })
action_space = spaces.Discrete(3)

model = MODEL_DEFAULTS.update({'fcnet_hiddens': [512, 512], 'fcnet_activation': 'relu'})

config = (
    PPOConfig()
    .rollouts(num_rollout_workers=4, num_envs_per_worker=1)\
    .training(train_batch_size=4000, gamma=0.99, model=model, lr=0.0004)\
    .environment(disable_env_checking=True)\
    .multi_agent(
        policies={
            "random": PolicySpec(policy_class=RandomActions),
            "Heuristic_10": (HeuristicPolicy, heuristic_observation_space, action_space, {'difficulty': 0}),
            "Heuristic_100": (HeuristicPolicy, heuristic_observation_space, action_space, {'difficulty': 1}),
            "SimpleHand": (PPOTorchPolicy, simple_hand_observation_space, action_space, {}),
            "learned": PolicySpec(
                config={}
            ),
        },
        policy_mapping_fn=select_policy,
        policies_to_train=['learned', 'SimpleHand'],
    )\
    .resources(num_gpus=0)\
    .framework('torch')
)
trainer = config.build(env="poker")


/Users/adamprice/Applications/anaconda3/envs/stocktake/lib/python3.8/site-packages/ray/_private/ray_option_utils.py:266: DeprecationWarning: Setting 'object_store_memory' for actors is deprecated since it doesn't actually reserve the required object store memory. Use object spilling that's enabled by default (https://docs.ray.io/en/releases-2.0.1/ray-core/objects/object-spilling.html) instead to bypass the object store memory size limitation.
  warnings.warn(
2022-11-08 17:21:02,575	INFO worker.py:1518 -- Started a local Ray instance.
(pid=79313) /Users/adamprice/Applications/anaconda3/envs/stocktake/lib/python3.8/site-packages/tensorflow/python/autograph/impl/api.py:22: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
(pid=79313)   import imp
(pid=79315) /Users/adamprice/Applications/anaconda3/envs/stocktake/lib/python3.8/site-packages/tensorflow/python/autograph/impl/api.py:22: DeprecationWarning: the imp mod

In [5]:
#!tensorboard --logdir=~/ray_results --host 0.0.0.0

In [6]:
for i in range(1000):
    trainer.train()

2022-11-08 17:21:14,366	WARNING deprecation.py:47 -- DeprecationWarning: `_get_slice_indices` has been deprecated. This will raise an error in the future!


In [ ]:
trainer.save("checkpoint/ppo_poker")